In [1]:
import numpy as np

# numpy settings
np.random.seed(1)
np.set_printoptions(suppress=True, linewidth = 150)

In [20]:
# MNIST dataset
from dataload import read_images_labels

k = 10 # k-hot value

# load train
training_images_filepath = './data/train-images-idx3-ubyte/train-images-idx3-ubyte'
training_labels_filepath = './data/train-labels-idx1-ubyte/train-labels-idx1-ubyte'
x_train, y_train = read_images_labels(training_images_filepath, training_labels_filepath)

# load test
test_images_filepath = './data/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'
test_labels_filepath = './data/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte'
x_test, y_test = read_images_labels(test_images_filepath, test_labels_filepath)

print(np.array(x_train[45854]).reshape(28,28))
print(np.array(y_train[45854]))

# reformat data for model
x_train = x_train.transpose() * (1/255)

# data dimension D
D = x_train.shape[0]
assert D==784
# number of training examples N
N = x_train.shape[1]

# reformat data to k-hot format TODO: does numpy have a better way to do this?
temp_array = np.zeros((k, N))
for index,val in enumerate(y_train):
    temp_array[val][index] = 1
print(temp_array[:, 45854])
y_train = temp_array

x_test = x_test.transpose() * (1/255)
y_test = y_test.reshape(1,-1)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  64 128 128 128 255 255 255 255 255 191  64   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  64 255 255 255 255 255 255 255 255 255 255 255 191   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 128 255 255 255 255 255 255 255 255 255 255 255 255  64   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  64 255 255 191 128 128   0 128 128 128 191 255 255 19

In [5]:
# iris dataset

k = 3 # k-hot value

# sepal_length,sepal_width,petal_length,petal_width,species
iris_dataset = np.genfromtxt('iris.csv', delimiter=',')

# randomize and split
np.random.seed(1)
np.random.shuffle(iris_dataset)

data = iris_dataset[:, [range(4)]]
labels = iris_dataset[:, 4]

data = np.reshape(data, (150,4))

x_train = data[[range(iris_dataset.shape[0]//5*3)]]
x_train = x_train.transpose()
x_test = data[[range(iris_dataset.shape[0]//5*3, iris_dataset.shape[0])]]
x_test = x_test.transpose()

# data dimension D
D = x_train.shape[0]
assert D==4
# number of training examples N
N = x_train.shape[1]

labels_train = labels[[range(iris_dataset.shape[0]//5*3)]]

# format
# reformat data to k-hot format TODO: does numpy have a better way to do this?
y_train = np.zeros((k, N))
for index, val in enumerate(labels_train): y_train[int(val)][index] = 1

y_test = labels[iris_dataset.shape[0]//5*3:iris_dataset.shape[0]].reshape(1,-1)


/tmp/ipykernel_2508/1994746260.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x_train = data[[range(iris_dataset.shape[0]//5*3)]]
/tmp/ipykernel_2508/1994746260.py:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  x_test = data[[range(iris_dataset.shape[0]//5*3, iris_dataset.shape[0])]]
/tmp/ipykernel_2508/1994746260.py:34: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, 

In [21]:
### instatiate model architecture

# architecture hyperparameters
hidden_dim = 15 # width of hidden layer

# instatiate first matrix of weights
w_2 = np.random.normal(loc=0, scale=1, size=(hidden_dim,D))
# instatiate first bias vector
b_2 = np.random.normal(loc=0, scale=1, size=(hidden_dim,1))

# instatiate second set of weights
w_3 = np.random.normal(loc=0, scale=1, size=(k,hidden_dim))
# instatiate second set of weights
b_3 = np.random.normal(loc=0, scale=1, size=(k,1))

def sigmoid(x): return 1/(1+np.exp(-x))

def sigmoid_prime(x): 
    sig = sigmoid(x)
    return sig * (1-sig)

# implementation of softmax from: https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python 
def softmax(x): 
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

### training

# training hyperparamters
alpha = 0.1 # learning rate
epochs = 300 # number of epochs
batch_size = 5

def extrem(array, array_name): return f'Extremal Values for {array_name}: {np.amin(array), np.amax(array)}'

def loss(activation, label): return 0.5*(activation-label)**2

def cost(loss_vec): return np.average(loss_vec)

for epoch in range(epochs):
    for batch_index in range(int(N/batch_size)): # look at this again
        x_train_batch = x_train[:, [batch_index, batch_index + batch_size]] 
        y_train_batch = y_train[:, [batch_index, batch_index + batch_size]]

        # forward pass
        z_2 = w_2.dot(x_train_batch) + b_2.dot(np.ones((1,x_train_batch.shape[1])))
        a_2 = sigmoid(z_2) 
        z_3 = w_3.dot(a_2) + b_3.dot(np.ones((1,a_2.shape[1])))
        a_3 = sigmoid(z_3)

        # backprop
        delta_3 = np.multiply((a_3-y_train_batch), sigmoid_prime(z_3))
        delta_2 = np.multiply(w_3.transpose().dot(delta_3), sigmoid_prime(z_2))
        # print(delta_2.shape)
        # print(x_train_batch.transpose().shape)

        # FULL gradient descent TODO: stochastic
        # make consistent layer labelling! Nielsen and Gross used different indexing for layer number
        w_3 -= alpha*(1/batch_size)*delta_3.dot(a_2.transpose())
        b_3 -= alpha*np.reshape(np.mean(delta_3, axis=1), (-1,1))


        w_2 -= alpha*(1/batch_size)*delta_2.dot(x_train_batch.transpose())
        b_2 -= alpha*np.reshape(np.mean(delta_2, axis=1), (-1,1))
        # if batch_index % 10 == 0: print(f"Batch {batch_index} completed.")

    # final foward pass
    a_2 = sigmoid(w_2.dot(x_train) + b_2.dot(np.ones((1,x_train.shape[1]))))
    a_3 = sigmoid(w_3.dot(a_2) + b_3.dot(np.ones((1,x_train.shape[1]))))

    loss_val = loss(a_3, y_train)
    cost_val = cost(loss_val)
    if epoch % 100 == 0: print(f"Cost at epoch {epoch}: {cost_val}")
    # print(f"Cost at epoch {epoch}: {cost_val}")


Cost at epoch 0: 0.03712917766447455
Cost at epoch 100: 0.007377760365864374
Cost at epoch 200: 0.007175502898641945
Cost at epoch 300: 0.007185704615716157


KeyboardInterrupt: 

In [22]:
### testing

# test dimension D_test
D_test = x_test.shape[1]

test_ones_1 = np.ones((1,D_test))
test_ones_2 = np.ones((1,D_test))

# forward pass on test data
test_a_2 = sigmoid(w_2.dot(x_test) + b_2.dot(np.ones((1,x_test.shape[1]))))
test_a_3 = sigmoid(w_3.dot(test_a_2) + b_3.dot(np.ones((1,x_test.shape[1]))))
predictions = np.argmax(test_a_3, axis=0)

num_wrong = np.sum(np.not_equal(y_test, predictions))
test_size = y_test.shape[1]
print(f"{num_wrong} of {test_size} wrong \nAccuracy: {1 - (num_wrong/test_size)}")

968 of 10000 wrong 
Accuracy: 0.9032


In [ ]:
# to compare to
import torch
import torch.nn

model = torch.nn.Sequential(
    torch.nn.Linear(784,hidden_dim, dtype=torch.float64),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_dim,10, dtype=torch.float64),
    torch.nn.Sigmoid()
)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

loss_fn = torch.nn.MSELoss()

x_train_tensor = torch.from_numpy(x_train.transpose())
y_train_tensor = torch.from_numpy(y_train.transpose())

for epoch in range(epochs):
    y_pred = model(x_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

x_test_tensor = torch.from_numpy(x_test.transpose())

y_test_t = model(x_test_tensor)
predictions = torch.argmax(y_test_t, dim=1)
